In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


import phik
from phik.report import plot_correlation_matrix
from phik import report

import sklearn
from sklearn.feature_selection  import SelectKBest, f_classif, f_regression, mutual_info_classif, SelectFpr
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, RobustScaler


from sklearn import metrics

from sklearn.model_selection import train_test_split

from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings("ignore")

from datetime import datetime, timedelta

from catboost import CatBoostClassifier
from catboost.utils import get_roc_curve

from catboost import cv
from catboost import Pool
from category_encoders import CatBoostEncoder, WOEEncoder, OneHotEncoder

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import classification_report

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.datasets import load_wine
import optuna
from optuna.samplers import TPESampler
import catboost
import pickle

In [2]:
pd.options.display.max_columns = 200
pd.options.display.max_rows = 200

# Данные анкеты

## загрузка

In [3]:
df= pd.read_csv("C:/Users/e8/Documents/new_scoring_23/anketa.csv", sep=';')

## обработка фичей

In [4]:
df = df[df['loan_cnt'].isin([0])]

In [5]:
df['OrganizationPhone'] = df['OrganizationPhone'].astype('str')

df['OrganizationPhone_group'] = df.apply(lambda x: 1 if '+7 (9' in x['OrganizationPhone'] else 2, axis = 1)

df['OrganizationPhone_group'] = df.apply(lambda x: 3 if '+7 (4' in x['OrganizationPhone']
                                                 else x['OrganizationPhone_group'], axis = 1)

df['OrganizationPhone_group'] = df.apply(lambda x: 4 if 'nan' in x['OrganizationPhone']
                                                 else x['OrganizationPhone_group'], axis = 1)

df['OrganizationPhone_group'].value_counts(dropna=False)

OrganizationPhone_group
1    367076
4    354164
2    270028
3    161447
Name: count, dtype: int64

In [6]:
del df['PersonId']
del df['OrganizationPhone']

# Данные доп

## Загрузка

In [7]:
df1= pd.read_csv("C:/Users/e8/Documents/new_scoring_23/data_.csv", on_bad_lines='skip', sep=';')

## Обработка и джойним

In [10]:
df1 = df1[df1['loan_cnt'].isin([0])]

del df1['InsuranceContract']

del df1['PersonId']
del df1['LoanId']

In [11]:
df = df1.merge(df, on='DraftId', how = 'left')

In [12]:
df['StoreName'] = df['StoreName'].fillna('website')

In [13]:
df = df.applymap(lambda x: x.upper() if isinstance(x, str) else x)

In [14]:
df['Bank'] = df['Bank'].apply(lambda x: 'SBER' if 'SBER' in str(x) else x)
df['Bank'] = df['Bank'].apply(lambda x: 'TINKOFF' if 'TINKOFF' in str(x) else x)
df['Bank'] = df['Bank'].apply(lambda x: 'ALFA-BANK' if 'ALFA-BANK' in str(x) else x)
df['Bank'] = df['Bank'].apply(lambda x: 'VTB' if 'VTB' in str(x) else x)
df['Bank'] = df['Bank'].apply(lambda x: 'RAIFFEISENBANK' if 'RAIFFEISENBANK' in str(x) else x)
df['Bank'] = df['Bank'].apply(lambda x: 'OTKRITIE' if 'OTKRITIE' in str(x) else x)
df['Bank'] = df['Bank'].apply(lambda x: 'QIWI' if 'QIWI' in str(x) else x)
df['Bank'] = df['Bank'].apply(lambda x: 'POCHTA BANK' if 'POCHTA' in str(x) else x)
df['Bank'] = df['Bank'].apply(lambda x: 'GAZPROMBANK' if 'GAZPROMBANK' in str(x) else x)
df['Bank'] = df['Bank'].apply(lambda x: 'PROMSVYAZBANK' if 'PROMSVYAZBANK' in str(x) else x)
df['Bank'] = df['Bank'].apply(lambda x: 'AGRICULTURAL BANK' if 'AGRICULTURAL BANK' in str(x) else x)
df['Bank'] = df['Bank'].apply(lambda x: 'AK BARS' if 'AK BARS' in str(x) else x)
df['Bank'] = df['Bank'].apply(lambda x: 'HOME' if 'HOME CREDIT' in str(x) else x)
df['Bank'] = df['Bank'].apply(lambda x: 'URALSIB' if 'URALSIB' in str(x) else x)
df['Bank'] = df['Bank'].apply(lambda x: 'ROSBANK' if 'ROSBANK' in str(x) else x)
df['Bank'] = df['Bank'].apply(lambda x: 'YANDEX' if 'YANDEX' in str(x) else x)
df['Bank'] = df['Bank'].apply(lambda x: 'UNICREDIT' if 'UNICREDIT' in str(x) else x)


In [17]:
df = df[df['Gender'].isin([1,2])]
df['Gender'] = df['Gender'] - 1
df['Gender'].value_counts()

Gender
0.0    647645
1.0    503806
Name: count, dtype: int64

In [18]:
df = df.drop_duplicates('DraftId')

In [19]:
df.columns

Index(['DraftId', 'CreateTime', 'InsurancePolicy', 'VerifiedByESIA',
       'EMailVerified', 'EMailToken', 'drafts_count', 'off_registered',
       'RegisterSameFact', 'Leads', 'Referer', 'StoreName', 'mobile',
       'PingTreeLead', 'Bin', 'Bank', 'mask_person_addcontact', 'card_cnt',
       'AttemptCount_sum', 'loan_cnt_x', 'Default_00', 'Default_30',
       'Default_90', 'num_model', 'LoanAmount', 'RequestAmount', 'age',
       'ActivityType', 'Education', 'Gender', 'LoanPurpose', 'MaritalStatus',
       'MonthlyIncome', 'Post', 'loan_cnt_y', 'OrganizationPhone_group'],
      dtype='object')

In [20]:
# оставляем нужные столбцы
df = df[['DraftId', 'CreateTime',  'InsurancePolicy',
       'VerifiedByESIA', 'EMailVerified', 'EMailToken', 'drafts_count',
        'RegisterSameFact',  'StoreName',
       'mobile', 'PingTreeLead', 'Bin', 'Bank', 'mask_person_addcontact',
        'age',
       'ActivityType', 'Education', 'Gender', 'LoanPurpose', 'MaritalStatus',
       'MonthlyIncome', 'Post', 'OrganizationPhone_group','LoanAmount',  'Default_00', 'Default_30',
       'Default_90']]

In [21]:
attribs=[ 'InsurancePolicy',
       'VerifiedByESIA', 'EMailVerified', 
       'RegisterSameFact',  'StoreName',
       'mobile', 'PingTreeLead', 'Bin', 'Bank', 'mask_person_addcontact',
         'ActivityType', 'Education', 'Gender', 'LoanPurpose', 'MaritalStatus',
       'MonthlyIncome', 'Post', 'OrganizationPhone_group']

In [22]:
df[attribs] = df[attribs].fillna('0')

In [23]:
# df.to_csv('original.csv', sep = ';') # сохраняем ориг если надо 

## Убираем хвосты

In [24]:
my_dict = {}
#обрабатываем категориальные переменные:
#  ЗДЕСЬ  обрезаем хвосты - признаки, которых мало объединяем в группу 'others'
if 1==1:
    '''attribs=[ 'InsurancePolicy',
       'VerifiedByESIA', 'EMailVerified', 
        'RegisterSameFact',  'StoreName',
       'mobile', 'PingTreeLead', 'Bin', 'Bank', 'mask_person_addcontact',
         'ActivityType', 'Education', 'Gender', 'LoanPurpose', 'MaritalStatus',
       'MonthlyIncome', 'Post', 'OrganizationPhone_group']'''                            
    NN=150    #   NN - минимальное количество ОСНОВНЫХ категорий, остальные слова в "хвосты" для обрубания     

    for attrib in attribs:       #   бежим по текстовым атрибиутам

        df[attrib]=df[attrib].astype(str)
        
        ll={}        #   имена частовстречающихся категорий
        t=[]         #   имена хвостов

        ii=0
        for i,j in df[attrib].value_counts().items():      
            ii=ii+1
    #    по основной базе формирую словарь ОСНОВНЫХ слов, затем в df_rs_fssp удаляю  ТО, ЧТО НЕ ПОПАЛО в основные слова     

            if j<NN:            # здесь подбираю вручную ----------------------  j сколько раз встретилось  ---------------------------
                t.append(i)              #  ФОрмирую словарь из "хвостов"
            else:
                
                ll.update({i:j})         #  формирую словарь с именами, которые буду заменять индивидуально

        print(attrib,"хвосты\n",t)                     #    нужно это учитыввать в МОДЕЛИ и ВЕСАХ  !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        print(attrib,"Что оставляем\n",ll,'\n',ll.keys())                     #    нужно это учитыввать в МОДЕЛИ и ВЕСАХ  !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        print(attrib,'\n',ll.keys())  
        my_dict[attrib] = list(ll.keys())
        df.loc[~df[attrib].isin(ll), attrib] = 'others'            # удаляю хвосты   
        
        #print(df[attrib].value_counts())  
         
    print('do it')





InsurancePolicy хвосты
 []
InsurancePolicy Что оставляем
 {'1': 775717, '0': 373972} 
 dict_keys(['1', '0'])
InsurancePolicy 
 dict_keys(['1', '0'])
VerifiedByESIA хвосты
 []
VerifiedByESIA Что оставляем
 {'0': 1105437, '1': 44252} 
 dict_keys(['0', '1'])
VerifiedByESIA 
 dict_keys(['0', '1'])
EMailVerified хвосты
 []
EMailVerified Что оставляем
 {'0': 1137969, '1': 11720} 
 dict_keys(['0', '1'])
EMailVerified 
 dict_keys(['0', '1'])
RegisterSameFact хвосты
 ['0']
RegisterSameFact Что оставляем
 {'1.0': 1077021, '0.0': 72664} 
 dict_keys(['1.0', '0.0'])
RegisterSameFact 
 dict_keys(['1.0', '0.0'])
StoreName хвосты
 []
StoreName Что оставляем
 {'WEBSITE': 1070650, 'PLAYMARKET': 54991, 'APPSTORE': 22067, 'APPGALLERY': 1981} 
 dict_keys(['WEBSITE', 'PLAYMARKET', 'APPSTORE', 'APPGALLERY'])
StoreName 
 dict_keys(['WEBSITE', 'PLAYMARKET', 'APPSTORE', 'APPGALLERY'])
mobile хвосты
 []
mobile Что оставляем
 {'0': 1070650, '1': 79039} 
 dict_keys(['0', '1'])
mobile 
 dict_keys(['0', '1'])
PingTr

mask_person_addcontact хвосты
 []
mask_person_addcontact Что оставляем
 {'0': 1147909, '1': 1780} 
 dict_keys(['0', '1'])
mask_person_addcontact 
 dict_keys(['0', '1'])
ActivityType хвосты
 []
ActivityType Что оставляем
 {'0': 1066235, 'ТОРГОВЛЯ': 20271, 'ПРОМЫШЛЕННОСТЬ': 12638, 'ДРУГОЕ': 9434, 'СЕРВИС И УСЛУГИ': 8713, 'СТРОИТЕЛЬСТВО И НЕДВИЖИМОСТЬ': 6949, 'ТРАНСПОРТ': 5221, 'ГОСУДАРСТВЕННАЯ СЛУЖБА': 4818, 'НАУКА И ОБРАЗОВАНИЕ': 2815, 'МЕДИЦИНА': 2431, 'АРМИЯ': 2093, 'ОХРАННАЯ ДЕЯТЕЛЬНОСТЬ': 1908, 'ДОБЫВАЮЩИЙ СЕКТОР': 1743, 'ФИНАНСЫ, БАНКИ, СТРАХОВАНИЕ, КОНСАЛТИНГ': 1628, 'ИНФОРМАЦИОННЫЕ ТЕХНОЛОГИИ/ТЕЛЕКОММУНИКАЦИИ': 1625, 'КУЛЬТУРА И ИСКУССТВО': 809, 'ТЭК': 358} 
 dict_keys(['0', 'ТОРГОВЛЯ', 'ПРОМЫШЛЕННОСТЬ', 'ДРУГОЕ', 'СЕРВИС И УСЛУГИ', 'СТРОИТЕЛЬСТВО И НЕДВИЖИМОСТЬ', 'ТРАНСПОРТ', 'ГОСУДАРСТВЕННАЯ СЛУЖБА', 'НАУКА И ОБРАЗОВАНИЕ', 'МЕДИЦИНА', 'АРМИЯ', 'ОХРАННАЯ ДЕЯТЕЛЬНОСТЬ', 'ДОБЫВАЮЩИЙ СЕКТОР', 'ФИНАНСЫ, БАНКИ, СТРАХОВАНИЕ, КОНСАЛТИНГ', 'ИНФОРМАЦИОННЫЕ ТЕХНОЛОГИИ/ТЕЛЕКОММУНИКАЦИИ', '

OrganizationPhone_group хвосты
 []
OrganizationPhone_group Что оставляем
 {'1.0': 366435, '4.0': 353897, '2.0': 268962, '3.0': 160395} 
 dict_keys(['1.0', '4.0', '2.0', '3.0'])
OrganizationPhone_group 
 dict_keys(['1.0', '4.0', '2.0', '3.0'])
do it


In [25]:
print(my_dict)

{'InsurancePolicy': ['1', '0'], 'VerifiedByESIA': ['0', '1'], 'EMailVerified': ['0', '1'], 'RegisterSameFact': ['1.0', '0.0'], 'StoreName': ['WEBSITE', 'PLAYMARKET', 'APPSTORE', 'APPGALLERY'], 'mobile': ['0', '1'], 'PingTreeLead': ['0', '1'], 'Bin': ['220220.0', '220070.0', '553691.0', '220024.0', '220015.0', '522860.0', '427938.0', '437772.0', '481776.0', '220028.0', '220029.0', '533669.0', '220073.0', '427616.0', '546998.0', '555949.0', '220432.0', '220077.0', '220030.0', '220027.0', '427638.0', '536829.0', '220002.0', '427432.0', '489347.0', '427640.0', '458443.0', '489049.0', '546940.0', '424917.0', '427630.0', '220003.0', '546938.0', '427655.0', '469395.0', '427667.0', '220001.0', '537965.0', '220038.0', '220431.0', '220424.0', '220412.0', '676907.0', '427660.0', '220053.0', '220341.0', '521324.0', '220039.0', '446915.0', '400680.0', '220019.0', '558620.0', '427606.0', '553609.0', '427662.0', '427652.0', '546955.0', '524602.0', '427631.0', '546960.0', '405870.0', '546930.0', '5280

In [26]:
# df.to_csv('original_obrabotka.csv', sep = ';') # сохраняем после удаления хвостов

# WoEEncoder

## раздеяем трейн тест

In [27]:
train = df[(df['CreateTime']<'2023-09-01')&(df['Default_30'].notna())]

In [28]:
test = df[(df['CreateTime']>='2023-09-01')&(df['CreateTime']<'2023-12-01')]

In [29]:
# столбцы которые оставляем в доп и анкетных фичах
cols_dop = ['DraftId', 'InsurancePolicy',
       'VerifiedByESIA', 'EMailVerified', 
        'RegisterSameFact',  'StoreName',
       'mobile', 'PingTreeLead', 'Bin', 'Bank', 'mask_person_addcontact',
       'Default_30']

cols_anketa  = ['DraftId',   'age', 'ActivityType',
       'Education', 'Gender', 'LoanPurpose', 'MaritalStatus', 'MonthlyIncome',
       'Post', 'OrganizationPhone_group', 'Default_30']

In [31]:
# столбцы категориальных признаков
attribs_anketa = ['ActivityType',
 'Education',
 'Gender',
 'LoanPurpose',
 'MaritalStatus',
 'MonthlyIncome',
 'Post',
 'OrganizationPhone_group']

attribs_dop = [
 'InsurancePolicy',
 'VerifiedByESIA',
 'EMailVerified',
 'RegisterSameFact',
 'StoreName',
 'mobile',
 'PingTreeLead',
 'Bin',
 'Bank',
 'mask_person_addcontact']

In [32]:
train_anketa = train[cols_anketa]
train_dop = train[cols_dop]

In [33]:
test_anketa = test[cols_anketa]
test_dop = test[cols_dop]

## Обучаем

In [34]:
from feature_engine.encoding import WoEEncoder

In [39]:
woe_encoder = WoEEncoder(
    variables=attribs_anketa,
    ignore_format=True,
    fill_value = 0.5
    
)

woe_encoder.fit(train_anketa[attribs_anketa], train_anketa['Default_30'])

train_woe = woe_encoder.transform(train_anketa[attribs_anketa])
test_woe = woe_encoder.transform(test_anketa[attribs_anketa])


test_woe[['DraftId', 'age', 'Default_30']] = test_anketa[['DraftId', 'age', 'Default_30']]
train_woe[['DraftId', 'age', 'Default_30']] = train_anketa[['DraftId', 'age', 'Default_30']]

test_woe = test_woe[cols_anketa]
train_woe = train_woe[cols_anketa]

del train_woe['Default_30']
del test_woe['Default_30']

train_woe.to_csv('anketa_train_woe.csv', sep = ';')
test_woe.to_csv('anketa_test_woe.csv', sep = ';')

with open("anketa_woe_encoder.pickle", "wb") as file:
    pickle.dump(woe_encoder, file)

In [43]:
woe_encoder = WoEEncoder(
    variables=attribs_dop,
    ignore_format=True,
    fill_value = 0.5
)

woe_encoder.fit(train_dop[attribs_dop], train_dop['Default_30'])

train_woe = woe_encoder.transform(train_dop[attribs_dop])
test_woe = woe_encoder.transform(test_dop[attribs_dop])

test_woe[['DraftId', 'Default_30']] = test_dop[['DraftId',  'Default_30']]
train_woe[['DraftId', 'Default_30']] = train_dop[['DraftId',  'Default_30']]

test_woe = test_woe[cols_dop]
train_woe = train_woe[cols_dop]

del train_woe['Default_30']
del test_woe['Default_30']

train_woe.to_csv('dop_train_woe.csv', sep = ';')
test_woe.to_csv('dop_test_woe.csv', sep = ';')

del train_anketa['Default_30']
del test_anketa['Default_30']

del train_dop['Default_30']
del test_dop['Default_30']


# оригиналы до кодирования
train_anketa.to_csv('anketa_train.csv', sep = ';')
test_anketa.to_csv('anketa_test.csv', sep = ';')

train_dop.to_csv('dop_train.csv', sep = ';')
test_dop.to_csv('dop_test.csv', sep = ';')

with open("dop_woe_encoder.pickle", "wb") as file:
    pickle.dump(woe_encoder, file)
